# Q-Learning - First steps

First steps to implement a DQN agent

In [1]:
import numpy as np
import sys
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, MaxNLocator
import ipywidgets as widgets
from ipyevents import Event
import time
import gymnasium as gym
import torch

In [2]:
sys.path.insert(1, str(Path("../..")))
import andreas2048
from andreas2048.game import *
from andreas2048 import gym2048
#env = gym.make("andreas_2048")
env = gym2048.Env2048()

In [3]:
raise RuntimeError("STOP")

RuntimeError: STOP

In [ ]:
for episode in range(100):
    obs, info = env.reset()
    done = False

    while not done:
        env.step()